In [1]:
import os
import boto3
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.applications.vgg19 import (VGG19, preprocess_input, decode_predictions)
from tensorflow.keras.applications import MobileNet
from config import aws_access_key_id, aws_secret_access_key 

In [2]:
# Set your AWS credentials and region
aws_access_key_id = aws_access_key_id
aws_secret_access_key = aws_secret_access_key 
region_name = "us-east-2"
bucket_name = "capstone-pill-images"
# train_prefix = "dataset/train/"
# validation_prefix = "dataset/valid/"
data_prefix = "data/data"


# Create an S3 client
s3 = boto3.client(
    's3',
     aws_access_key_id=aws_access_key_id,
     aws_secret_access_key=aws_secret_access_key,
     region_name=region_name
    )

In [3]:

# Function to download images from S3 bucket to a local directory with subdirectories
def download_images_from_s3(bucket_name, prefix, local_directory):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    for obj in response.get('Contents', []):
        key = obj['Key']

        # Extract the drug name from the key
        # Split the key at the second "/" to get the correct drug name

    for obj in response.get('Contents', []):
        key = obj['Key']
        local_path = os.path.join(local_directory, os.path.relpath(key, prefix))  # Use os.path.relpath to get the relative path
        os.makedirs(os.path.dirname(local_path), exist_ok=True)  # Create intermediate directories if they don't exist
        s3.download_file(bucket_name, key, local_path)





In [4]:
# Download training images
local_drug_directory = "local_drug_directory"
download_images_from_s3(bucket_name, data_prefix, local_drug_directory)


In [5]:
# define loading parameters
img_height = 224
img_width = 224
batch_size = 1

In [6]:
import splitfolders
splitfolders.ratio('local_drug_directory', output="output", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 0 files [00:00, ? files/s]

Copying files: 552 files [00:01, 395.93 files/s]


In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_dir = "output/train/"
valid_dir = "output/val/"

In [9]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_width,img_height),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode="categorical"
                                                   )

Found 437 images belonging to 23 classes.


In [10]:
validation_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    target_size=(img_width,img_height),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    class_mode="categorical"
                                                   )

Found 46 images belonging to 23 classes.


In [11]:
class_names = train_generator.class_indices.keys()
class_names

dict_keys(['Amoxicillin 500 mg', 'Apixaban 2.5 mg', 'Aprepitant 80 mg', 'Atomoxetine 25 mg', 'Calcitriol 0.00025', 'Prasugrel 10 MG', 'Ramipril 5 MG', 'Saxagliptin 5 MG', 'Sitagliptin 50 MG', 'Tadalafil 5 MG', 'carvedilol 3.125', 'celecoxib 200', 'duloxetine 30', 'eltrombopag 25', 'metformin_500', 'montelukast-10', 'mycophenolate-250', 'omeprazole_40', 'oseltamivir-45', 'pantaprazole-40', 'pitavastatin_1', 'prednisone_5', 'sertraline_25'])

In [12]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D

In [13]:
base_model = MobileNet(weights = 'imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32

In [14]:
model = Sequential()

model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(23, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functi  (None, 7, 7, 1024)        3228864   
 onal)                                                           
                                                                 
 global_average_pooling2d (  (None, 1024)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 23)                23575     
                                                                 
Total params: 3252439 (12.41 MB)
Trainable params: 23575 (92.09 KB)
Non-trainable params: 3228864 (12.32 MB)
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [16]:
model.fit(train_generator, epochs=2)

Epoch 1/2
437/437 [==============================] - 10s 20ms/step - loss: 0.9893 - accuracy: 0.7941
Epoch 2/2
437/437 [==============================] - 9s 22ms/step - loss: 0.0554 - accuracy: 0.9977


In [17]:
model_loss, model_accuracy = model.evaluate(validation_generator, verbose=2)
print(f" loss : {round(model_loss)*100}%, accuray : {round(model_accuracy,2)*100}%")

46/46 - 2s - loss: 0.0218 - accuracy: 1.0000 - 2s/epoch - 34ms/step
 loss : 0%, accuray : 100.0%


In [20]:
model.save('MobileNet_02.keras')